## 通道注意力

**通道注意力是什么?**

通道注意力通过对各通道进行加权组合，从整体上突出重要通道、抑制次要通道；若从单个通道角度来看，其效果可等价为一次仿射变换（缩放与平移）。



**不同任务下的注意力机制含义?**

- NLP 任务：主要在序列维度上建模（词与词之间的关系），注意力机制用于突出文本序列中关键的 token。

- 图像任务：注意力可以作用于不同维度
    - 通道注意力：强调重要通道的信息，抑制次要通道。
    - 空间注意力：突出不同 patch 或像素的位置重要性。




**位置编码问题**

- 文本注意力：一维位置编码是必须的，用于保留序列顺序信息。

- 图像空间注意力：二维位置编码通常是必须的，尤其在全局注意力或 Transformer 架构中，用于保留像素或 patch 的空间关系。

- 图像通道注意力：输入通常是经过卷积提取的特征图，已经隐含空间信息，因此不需要额外的位置编码。


下面代码是通道注意力的实现



In [ ]:

import math
import torch
import torch.nn as nn
import numpy as np
from einops import rearrange
from icecream import ic
def Normalize(in_channels, num_groups=32):
    return torch.nn.GroupNorm(num_groups=num_groups, num_channels=in_channels, eps=1e-6, affine=True)


def nonlinearity(x):
    # swish
    return x*torch.sigmoid(x)

class AttnBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = Normalize(in_channels)
        self.q = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.k = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.v = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.proj_out = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)


    def forward(self, x):
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)
        ic(q.shape)

        # compute attention
        b,c,h,w = q.shape
        q = q.reshape(b,c,h*w)
        q = q.permute(0,2,1)   # b,hw,c
        k = k.reshape(b,c,h*w) # b,c,hw
        ic(q.shape)
        w_ = torch.bmm(q,k)     # b,hw,hw    w[b,i,j]=sum_c q[b,i,c]k[b,c,j]
        ic(w_.shape)
        w_ = w_ * (int(c)**(-0.5))
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = v.reshape(b,c,h*w)
        w_ = w_.permute(0,2,1)   # b,hw,hw (first hw of k, second of q)
        h_ = torch.bmm(v,w_)     # b, c,hw (hw of q) h_[b,c,j] = sum_i v[b,c,i] w_[b,i,j]
        h_ = h_.reshape(b,c,h,w)

        h_ = self.proj_out(h_)

        return x+h_


img=torch.rand(2,512,16,16)
attn=AttnBlock(img.shape[1])

out=attn(img)
ic(out.shape)


ic| q.shape: torch.Size([2, 512, 16, 16])
ic| q.shape: torch.Size([2, 256, 512])
ic| w_.shape: torch.Size([2, 256, 256])
ic| out.shape: torch.Size([2, 512, 16, 16])


torch.Size([2, 512, 16, 16])